### Korean OCR with PaddleOCR

테스트: PaddleOCR 를 이용하여 모든 텍스트를 얻어내고 llm 을 통해 데이타 구조화를 시도한다. 

In [22]:
import sys
import os
import io
import paddleocr
import base64

# OCR 엔진
from paddleocr import PaddleOCR
import inspect

# 이미지/과학 계산
import numpy as np
from PIL import Image as PILImage
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 내부 모듈 - 이미지 전처리 함수들
try:
    from app.services.analysis.image_preprocessing import (
        # 공통 함수들
        pdf_to_images, open_with_exif, save_png_bytes, apply_pipeline,
        
        # 전처리 함수들
        flatten_transparency, auto_crop_with_margin, normalize_mode,
        upscale_min_resolution, illumination_flatten, suppress_glare,
        weak_autocontrast, apply_clahe, conservative_sharpen, blacken_reddish_text, blacken_bluish_text, to_grayscale,
        adaptive_binarize_for_ocr, enhance_table_lines, table_smart_crop,
        add_white_border, downscale_target_long_edge, ocr_quality_gate,
        
        # 고급 전처리 함수들 (필요시)
        detect_document_quad, perspective_unwarp, deskew_textlines,
        conditional_dewarp
    )
    print("✅ 이미지 전처리 함수들 import 완료!")
except ImportError as e:
    print(f"❌ Import 오류: {e}")
    print("📂 현재 sys.path:", sys.path[-3:])  

# 프로젝트 루트 경로 추가
sys.path.append('/home/aidan/work/meow-chat')

# 환경 정보 확인
print(f"🐍 Python 경로: {sys.executable}")
print(f"📁 작업 디렉토리: {os.getcwd()}")

# PATH 확인 (처음 3개만)
path_list = os.environ['PATH'].split(':')
print(f"\n📋 PATH 순서:")
for i, path in enumerate(path_list[:3], 1):
    marker = "✅" if "meow-chat" in path else "📍"
    print(f"  {i}. {marker} {path}")

print(f"\n🔍 현재 PaddleOCR 버전: {paddleocr.__version__}")

# PaddleOCR 생성자의 파라미터 확인
signature = inspect.signature(PaddleOCR.__init__)
print(f"\n📋 지원하는 파라미터:")
for param_name, param in signature.parameters.items():
    if param_name != 'self':
        default = param.default if param.default != inspect.Parameter.empty else "필수"
        print(f"   {param_name}: {default}")      

✅ 이미지 전처리 함수들 import 완료!
🐍 Python 경로: /home/aidan/miniconda3/envs/meow-chat/bin/python
📁 작업 디렉토리: /home/aidan/work/meow-chat

📋 PATH 순서:
  1. ✅ /home/aidan/miniconda3/envs/meow-chat/bin
  2. 📍 /home/aidan/.vscode-server/bin/7d842fb85a0275a4a8e4d7e040d2625abbf7f084/bin/remote-cli
  3. 📍 /home/aidan/.local/bin

🔍 현재 PaddleOCR 버전: 3.2.0

📋 지원하는 파라미터:
   doc_orientation_classify_model_name: None
   doc_orientation_classify_model_dir: None
   doc_unwarping_model_name: None
   doc_unwarping_model_dir: None
   text_detection_model_name: None
   text_detection_model_dir: None
   textline_orientation_model_name: None
   textline_orientation_model_dir: None
   textline_orientation_batch_size: None
   text_recognition_model_name: None
   text_recognition_model_dir: None
   text_recognition_batch_size: None
   use_doc_orientation_classify: None
   use_doc_unwarping: None
   use_textline_orientation: None
   text_det_limit_side_len: None
   text_det_limit_type: None
   text_det_thresh: None
   text

In [23]:
# 환경변수 로드 (.env 파일은 프로젝트 루트에 위치)
load_dotenv()  # 현재 디렉토리와 상위 디렉토리에서 자동으로 .env 파일을 찾음

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 환경 확인
print(f"🔑 API Key 설정: {'✅ 완료' if os.getenv('OPENAI_API_KEY') else '❌ 없음'}")
print(f"📂 현재 경로: {os.getcwd()}")
print(f"📄 PDF 파일 존재: {'✅ 있음' if os.path.exists('../../test/data/은내과.pdf') else '❌ 없음'}")

🔑 API Key 설정: ✅ 완료
📂 현재 경로: /home/aidan/work/meow-chat
📄 PDF 파일 존재: ❌ 없음


### 1. OCR 인스턴스 생성

In [24]:
from app.services.ocr.paddle_ocr import PaddleOCRService

# 서버급 조합(정확도 지향): PP-OCRv5_server_det + korean_PP-OCRv5_server_rec
# 필요 시 파라미터는 추가로 조정 가능: text_det_limit_side_len, text_det_thresh, text_rec_score_thresh 등
ocr = PaddleOCRService(
    lang="korean",
)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)

In [25]:
# ocr.get_available_langs()

In [26]:
# ocr.get_available_models()

In [27]:
# ocr.get_current_model_info()
info = ocr.get_current_model_info()
print(info)

🔍 현재 사용 중인 모델 정보 (실제 설정)
📌 설정 언어: korean
📌 파이프라인: OCR
📌 텍스트 유형: general
📌 모델 버전: PP-OCRv5

🔍 핵심 모델:
   텍스트 감지: PP-OCRv5_server_det
   텍스트 인식: korean_PP-OCRv5_mobile_rec

📐 문서 전처리 모델:
   PP-LCNet_x1_0_doc_ori (방향 보정)
   UVDoc (문서 교정)

📏 텍스트라인 방향 보정:
   PP-LCNet_x1_0_textline_ori

⚙️  파이프라인 설정:
   문서 전처리: ✅
   텍스트라인 보정: ✅

🔍 감지 모델 설정:
   임계값: 0.3
   박스 임계값: 0.6
   언클립 비율: 1.5

📝 인식 모델 설정:
   배치 크기: 6
   점수 임계값: 0.0

📊 성능 특성 (실제 설정 기반):
   모델 조합: 서버급 감지 + 모바일 인식
   한국어 최적화: ✅
   실시간 처리: ✅
   문서 처리: ✅ 고급
{'language': 'korean', 'version': 'PP-OCRv5', 'detection_model': 'PP-OCRv5_server_det', 'recognition_model': 'korean_PP-OCRv5_mobile_rec', 'doc_orientation_models': ['PP-LCNet_x1_0_doc_ori', 'UVDoc'], 'textline_orientation_model': 'PP-LCNet_x1_0_textline_ori', 'pipeline_config': {'use_doc_preprocessor': True, 'use_textline_orientation': True, 'text_type': 'general'}}


In [28]:
# ocr.get_current_preprocessing_settings()

In [29]:
# img_path = os.path.abspath('notebooks/ocr/assets/images/test_image_1.jpg')

# img_path = os.path.abspath('notebooks/ocr/assets/images/20241106.jpg')   # ---------- 날짜가 여러개
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241107_2.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241107.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241121_2.png')  # ---------- 날짜 없는것
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241121.png')    # ---------- 사진으로 짝어서 삐뚤어진것 값 안나옴. 
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241123_2.png')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241123.png')    # ---------- 인쇄일자만 있음
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241205_2.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241205.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241219.png')    # ---------- 사진으로 짝어서 삐뚤어진것
img_path = os.path.abspath('notebooks/ocr/assets/images/20241221_2.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241221.jpg')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241224_2.png')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241224.png')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20241231.png')
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250107.png')    # ----------
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250110.png')    # ---------- 데이블 구조 특이한것. value 열 레이블이 날짜임
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250116_2.png')  
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250116_3.png')  # ---------- blue 글자 인식이 잘 안됨
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250116_4.png') 
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250116.png') 
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250119_2.png')  # ---------- 흐릿한 이미지
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250119_3.png') 
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250119.png')    # ---------- 흐릿한 이미지
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250125.jpeg') # ----------   A4 지 1장 이상 분량은 대량 오류 RDW-CV  RETIC#
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250203.jpeg') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250207.jpeg') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250209.jpeg') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250216.jpeg') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250222.png') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250228.png') # ----------   A4 지 1장 이상 분량은 대량 오류
# img_path = os.path.abspath('notebooks/ocr/assets/images/20250301.png')



# img_path = os.path.abspath('notebooks/ocr/assets/images/20250125.jpeg')



print(f"분석할 이미지: {img_path}")    

분석할 이미지: /home/aidan/work/meow-chat/notebooks/ocr/assets/images/20241221_2.jpg


In [30]:
def preprocess(
    img_bytes: bytes,    
    debug: bool = True,
) -> bytes:
    try:
        if debug:
            print("🏥 의료 문서용 전처리 시작")
        
        img = open_with_exif(img_bytes) # EXIF 회전 교정
        original_size = img.size
        
        if debug:
            print(f"📏 원본 이미지 크기: {original_size}")
        
        steps = [
            # 기본 정리 (필수 선행 단계)
            (flatten_transparency, {}),              # 투명도 제거
            (normalize_mode, {}),                    # 모드 표준화 (L/RGB)
        
            # 해상도 표준화 (최소 품질 확보)
            (upscale_min_resolution, {}),            
            
            # 빛반사 제거
            (suppress_glare, {}),                   
            
            # 대비 향상
            (weak_autocontrast, {}),               
            
            # 색 텍스트 보정(있으면)
            (blacken_reddish_text, {}),  
            (blacken_bluish_text, {}), 

            # 회색화
            (to_grayscale, {}),    

            # 기하학적 보정 (픽셀 수준 작업)
            # (auto_crop_with_margin, {}),           # 여백 제거
            (conditional_dewarp, {}),                # 페이지 휘어짐 보정 
            (deskew_textlines, {}),                  # 미세 기울기 보정               

            # 표 라인 강화
            (enhance_table_lines, {}), 
            
            # 최종 크기 조정 (토큰 절약)
            (downscale_target_long_edge, {}),      

            # 선명도 향상
            (conservative_sharpen, {}),           
        ]
    
        # 파이프라인 실행
        img = apply_pipeline(img, steps)
        
        # PNG로 저장
        buf = io.BytesIO()
        if img.mode == 'L':  # 그레이스케일
            img.save(buf, format="JPEG", quality=85, optimize=True)
        else:  # RGB
            img.save(buf, format="JPEG", quality=85, optimize=True, subsampling=2)
        result_bytes = buf.getvalue()
        
        if debug:
            final_size = img.size
            print(f"📐 최종 이미지 크기: {final_size}")
            print(f"🔄 크기 변화: {original_size} → {final_size}")
            # 용량 변화 및 퍼센트 계산
            original_bytes = len(img_bytes)
            result_bytes_len = len(result_bytes)
            percentage = (result_bytes_len / original_bytes) * 100
            
            print(f"💾 용량 변화: {original_bytes:,} bytes → {result_bytes_len:,} bytes ({percentage:.1f}%)")
            print("✅ 전처리 완료")
        
        return result_bytes
        
    except Exception as e:
        if debug:
            print(f"❌ 전처리 실패: {e}")
            import traceback
            traceback.print_exc()
        
        print(f"⚠️ 전처리 실패, 원본 반환: {e}")
        return img_bytes 

In [31]:
# 이미지 파일을 읽고 전처리하여 바이트로 변환
with open(img_path, 'rb') as f:
    _raw_bytes = f.read()

_pre_bytes = preprocess(_raw_bytes, debug=True)

🏥 의료 문서용 전처리 시작
📏 원본 이미지 크기: (794, 1123)
[conditional_dewarp] r2_lin=0.157, r2_quad=0.157, gain=0.000, amp=1392.11px
[conditional_dewarp] 곡률/적합 품질 부족 - 스킵
[deskew_textlines] Tesseract OSD angle: -0.00°
[deskew_textlines] Hough angle: 0.00°
[deskew_textlines] coarse=-0.00°, final=0.00°
📐 최종 이미지 크기: (1358, 1920)
🔄 크기 변화: (794, 1123) → (1358, 1920)
💾 용량 변화: 69,559 bytes → 115,716 bytes (166.4%)
✅ 전처리 완료
[deskew_textlines] Tesseract OSD angle: -0.00°
[deskew_textlines] Hough angle: 0.00°
[deskew_textlines] coarse=-0.00°, final=0.00°
📐 최종 이미지 크기: (1358, 1920)
🔄 크기 변화: (794, 1123) → (1358, 1920)
💾 용량 변화: 69,559 bytes → 115,716 bytes (166.4%)
✅ 전처리 완료


In [32]:
# bytes를 PIL Image로 변환
# processed_img = PILImage.open(io.BytesIO(_pre_bytes))

# # 원본과 전처리된 이미지를 함께 표시
# fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# # 원본 이미지
# original_img = PILImage.open(img_path)
# axes[0].imshow(original_img)
# axes[0].set_title('original_img', fontsize=14)
# axes[0].axis('off')

# # 전처리된 이미지
# if processed_img.mode == 'L':  # 그레이스케일
#     axes[1].imshow(processed_img, cmap='gray', vmin=0, vmax=255)
# else:  # 컬러
#     axes[1].imshow(processed_img)
# axes[1].set_title('processed_img', fontsize=14)
# axes[1].axis('off')

# plt.tight_layout()
# plt.show()

In [33]:
# OCR 실행
print(f"🔍 OCR 분석 시작: {img_path}")
try:
    result = ocr.run_ocr_from_bytes(_pre_bytes)
    print("✅ OCR 처리 완료!")
except Exception as e:
    print(f"❌ OCR 처리 중 오류 발생: {e}")
    print(f"오류 타입: {type(e).__name__}")

🔍 OCR 분석 시작: /home/aidan/work/meow-chat/notebooks/ocr/assets/images/20241221_2.jpg
✅ OCR 처리 완료!
✅ OCR 처리 완료!


In [34]:
# OCR 결과 출력 - 새로운 형태로 테스트
# print(result)

ocr_data = ocr.extract_text_with_confidence(result[0])
print(f" ocr_data {ocr_data}")
ocr_data = ocr.postprocess_split_value_unit(ocr_data)
print(f" ocr_data {ocr_data}")

print("📋 OCR 결과:")
for i, item in enumerate(ocr_data, 1):
    print(f"  {i}. {item}")

print(f"\n총 {len(ocr_data)}개 텍스트 인식됨")

# json_string = ocr.convert_to_structured_json(result)
# print("📄 convert_to_structured_json 결과:")
# print(json_string)

 ocr_data [{'text': 'BAEK', 'confidence': 0.84}, {'text': '울', 'confidence': 0.54}, {'text': '백산동물병원', 'confidence': 0.98}, {'text': '진료기록-아전경(양희)', 'confidence': 0.92}, {'text': 'SAN', 'confidence': 0.99}, {'text': '서울특별시 강남구 역삼동725-15 Tel) 02-1644-5653', 'confidence': 0.94}, {'text': '8.64', 'confidence': 1.0}, {'text': '%', 'confidence': 1.0}, {'text': 'LYMPHO(%)', 'confidence': 0.99}, {'text': '2.496', 'confidence': 1.0}, {'text': '%', 'confidence': 0.99}, {'text': 'Mono(%)', 'confidence': 0.99}, {'text': 'EOS(%)', 'confidence': 0.96}, {'text': '0.962', 'confidence': 0.95}, {'text': '%', 'confidence': 1.0}, {'text': 'BASO (%)', 'confidence': 0.96}, {'text': '%', 'confidence': 1.0}, {'text': 'NEUT', 'confidence': 1.0}, {'text': '1.15 - 10.29', 'confidence': 0.94}, {'text': '18.065 KuL', 'confidence': 0.97}, {'text': 'LYMPH', 'confidence': 0.98}, {'text': '0.92-6.88', 'confidence': 0.99}, {'text': '1.78 KuL', 'confidence': 0.98}, {'text': '2', 'confidence': 0.13}, {'text': 'MONO', 'c

In [ ]:
import re
import json
from typing import List, Dict, Any, Optional, Tuple

# LLM 전용 파서: LLM이 토큰 인덱스만 반환하고, 실제 값은 반드시 ocr_data의 텍스트/컨피던스로부터 재구성합니다.
# - value는 항상 수치형이어야 하며, 값 끝의 H/L/N/High/Low/Normal 접미는 제거 후 숫자만 추출합니다.
# - code_confidence는 ocr_data의 해당 code 토큰의 confidence를 문자열로 그대로 사용합니다(반올림/포맷 변경 없음).
# - reference 역시 ocr_data의 텍스트로부터만 추출/가공합니다(임의 생성 금지).

_num_re = re.compile(r"[-+]?\d+(?:\.\d+)?")
_range_re = re.compile(r"([-+]?\d+(?:\.\d+)?)\s*[-–~]\s*([-+]?\d+(?:\.\d+)?)")
_suffix_re = re.compile(r"\s*(?:\(|\[)?\s*(?i:(?:high|low|normal)|h|l|n)\s*(?:\)|\])?\s*$")


def _extract_numeric(text: str) -> Optional[str]:
    if not text:
        return None
    t = text.strip()
    # 접미(H/L/N/High/Low/Normal) 제거
    t = _suffix_re.sub("", t)
    m = _num_re.search(t)
    return m.group(0) if m else None


def _extract_range(text: str) -> Tuple[Optional[str], Optional[str]]:
    if not text:
        return None, None
    m = _range_re.search(text)
    if not m:
        return None, None
    return m.group(1), m.group(2)


def _safe_get_token(tokens: List[Dict[str, Any]], idx: Optional[int]) -> Optional[Dict[str, Any]]:
    if idx is None:
        return None
    if not isinstance(idx, int):
        return None
    if idx < 0 or idx >= len(tokens):
        return None
    return tokens[idx]


def _build_tests_from_indices(tokens: List[Dict[str, Any]], plan: Dict[str, Any]) -> Dict[str, Any]:
    out = []
    tests = plan.get("tests", []) if isinstance(plan, dict) else []
    for t in tests:
        code_idx = t.get("code_idx")
        value_idx = t.get("value_idx")
        unit_idx = t.get("unit_idx")
        ref_min_idx = t.get("ref_min_idx")
        ref_max_idx = t.get("ref_max_idx")
        range_idx = t.get("range_idx")

        tok_code = _safe_get_token(tokens, code_idx)
        tok_value = _safe_get_token(tokens, value_idx)
        tok_unit = _safe_get_token(tokens, unit_idx)
        tok_ref_min = _safe_get_token(tokens, ref_min_idx)
        tok_ref_max = _safe_get_token(tokens, ref_max_idx)
        tok_range = _safe_get_token(tokens, range_idx)

        code_text = (tok_code or {}).get("text") or "UNKNOWN"
        code_confidence = str((tok_code or {}).get("confidence") if tok_code and "confidence" in tok_code else "")

        # value: 반드시 수치형, 접미(H/L/N/High/Low/Normal) 제거 후 숫자만
        raw_value_text = (tok_value or {}).get("text") if tok_value else None
        value_num = _extract_numeric(raw_value_text or "")
        if value_num is None:
            value_num = "UNKNOWN"
        # value_confidence: value 토큰의 confidence 그대로 문자열화
        value_confidence = str((tok_value or {}).get("confidence") if tok_value and "confidence" in tok_value else "")

        # unit: 공백으로 나뉘는 경우 마지막 토큰을 단위로 해석 (unit_idx에서만 추출)
        unit_text = "UNKNOWN"
        unit_confidence = "UNKNOWN"
        if tok_unit:
            ut = (tok_unit or {}).get("text") or ""
            ut = ut.strip()
            if ut:
                unit_text = ut.split()[-1]
            unit_confidence = str((tok_unit or {}).get("confidence") if "confidence" in tok_unit else "") or "UNKNOWN"

        # reference: ref_min/ref_max 인덱스가 우선, 없으면 range_idx에서 파싱
        ref_min = _extract_numeric((tok_ref_min or {}).get("text") or "") if tok_ref_min else None
        ref_max = _extract_numeric((tok_ref_max or {}).get("text") or "") if tok_ref_max else None
        ref_min_source = "min" if (tok_ref_min and ref_min is not None) else None
        ref_max_source = "max" if (tok_ref_max and ref_max is not None) else None
        if (ref_min is None or ref_max is None) and tok_range:
            rmin, rmax = _extract_range((tok_range or {}).get("text") or "")
            if ref_min is None and rmin is not None:
                ref_min = rmin
                ref_min_source = "range"
            if ref_max is None and rmax is not None:
                ref_max = rmax
                ref_max_source = "range"
        # 최종 값 보정 및 confidence 매핑
        if ref_min is None:
            ref_min = "UNKNOWN"
        if ref_max is None:
            ref_max = "UNKNOWN"
        if ref_min_source == "min" and tok_ref_min:
            ref_min_confidence = str((tok_ref_min or {}).get("confidence"))
        elif ref_min_source == "range" and tok_range:
            ref_min_confidence = str((tok_range or {}).get("confidence"))
        else:
            ref_min_confidence = "UNKNOWN"
        if ref_max_source == "max" and tok_ref_max:
            ref_max_confidence = str((tok_ref_max or {}).get("confidence"))
        elif ref_max_source == "range" and tok_range:
            ref_max_confidence = str((tok_range or {}).get("confidence"))
        else:
            ref_max_confidence = "UNKNOWN"

        out.append({
            "code": str(code_text),
            "code_confidence": code_confidence if code_confidence != "" else "UNKNOWN",
            "value": str(value_num),
            "value_confidence": value_confidence if value_confidence != "" else "UNKNOWN",
            "reference_min": str(ref_min),
            "reference_min_confidence": ref_min_confidence,
            "reference_max": str(ref_max),
            "reference_max_confidence": ref_max_confidence,
            "unit": str(unit_text),
            "unit_confidence": unit_confidence if unit_confidence != "" else "UNKNOWN",
        })
    return {"tests": out}


def analyze_medical_ocr_data_with_llm(ocr_data: List[Dict[str, Any]], model: str = "gpt-4.1-mini", seed: Optional[int] = 42) -> Dict[str, Any]:
    """
    LLM만을 사용해 다양한 형태의 OCR 토큰으로부터 테이블(검사코드/값/참고범위/단위)을 추출합니다.
    - LLM은 반드시 tokens 배열의 인덱스만 반환합니다(문자열 값을 직접 생성 금지).
    - value는 수치형만 허용하며, 접미 H/L/N/High/Low/Normal은 무시 후 숫자만 사용합니다.
    - code_confidence는 ocr_data의 해당 code 토큰 confidence를 문자열로 그대로 사용합니다.
    - model: 사용할 OpenAI 모델 이름(기본값 "gpt-4.1-mini").
    - seed: 가능하면 동일 결과 재현을 위한 시드(모델/SDK가 지원하는 경우에만 적용).
    반환: {"tests":[{code, code_confidence, value, value_confidence, reference_min, reference_min_confidence, reference_max, reference_max_confidence, unit, unit_confidence}]}
    """
    tokens_for_llm = [
        {"idx": i, "text": str(x.get("text", "")), "confidence": float(x.get("confidence", 0.0))}
        for i, x in enumerate(ocr_data)
        if str(x.get("text", "")).strip()
    ]
    print(f"tokens_for_llm: {len(tokens_for_llm)} items")

    system_prompt = (
        "당신은 반려동물 검사결과지 OCR 토큰들로부터 테이블 데이터를 구조화하는 전문가입니다.\n"
        "다음 지침을 반드시 지키세요:\n"
        "1) 다양한 레이아웃 처리: name/result/unit/reference, name/reference/result/unit 등 순서가 달라도 토큰 인접성과 형상을 활용해 대응.\n"
        "2) 컬럼 헤더가 보이면 헤더를 근거로 매핑, 없으면 숫자/범위/단위 패턴과 행 그룹을 추정.\n"
        "3) 참고범위는 'min-max' 하나의 토큰이거나, min/max가 분리될 수 있음.\n"
        "4) 반환 시 절대 문자열 값(코드/값/단위/범위)을 직접 생성하지 말고, 제공된 tokens 배열의 인덱스만 사용.\n"
        "5) 결과 포맷은 아래와 같음: {\"tests\":[{\"code_idx\":int,\"value_idx\":int|null,\"unit_idx\":int|null,\"ref_min_idx\":int|null,\"ref_max_idx\":int|null,\"range_idx\":int|null}]}.\n"
        "6) 인덱스는 tokens[idx]를 가리켜야 하며 유효하지 않은 인덱스를 생성하지 마세요."
    )

    user_prompt = (
        "아래 tokens(JSON 배열, 각 원소는 {idx,text,confidence})만을 이용해 검사 항목을 추출하세요.\n"
        "- 문자열 값을 직접 만들지 말고 인덱스만 반환하세요.\n"
        "- range가 하나의 토큰에 min-max로 있으면 range_idx만 채우고 ref_min_idx/ref_max_idx는 null 가능.\n"
        "- 분리되어 있으면 ref_min_idx/ref_max_idx로 각각 지정.\n"
        "- unit은 별도 토큰이면 unit_idx, 그렇지 않으면 null."
    )

    # Pylance 친화적: content를 단일 문자열로 합치고, seed는 extra_body로 전달
    content_str = user_prompt + "\n" + json.dumps({"tokens": tokens_for_llm}, ensure_ascii=False)

    payload: Dict[str, Any] = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content_str},
        ],
        "temperature": 0,
        "response_format": {"type": "json_object"},
    }
    if seed is not None:
        payload["extra_body"] = {"seed": seed}

    response = client.chat.completions.create(**payload)

    plan = json.loads(response.choices[0].message.content)
    print(f"plan {model} tests={len(plan.get('tests', []))}")
    return _build_tests_from_indices(tokens_for_llm, plan)

print("🧠 LLM 전용(인덱스 기반) 파서: model 파라미터 + seed(extra_body) 방식 적용!")

🧠 LLM 전용(인덱스 기반) 파서: model 파라미터 + seed 추가 완료!


In [36]:
# Quick mock validation of confidence fields (no API calls)
# _mock_tokens = [
#     {"idx": 0, "text": "LYMPHO(%)", "confidence": 0.98},
#     {"idx": 1, "text": "8.64", "confidence": 1.0},
#     {"idx": 2, "text": "%", "confidence": 1.0},
# ]
# _mock_plan = {"tests": [{
#     "code_idx": 0,
#     "value_idx": 1,
#     "unit_idx": 2,
#     "ref_min_idx": None,
#     "ref_max_idx": None,
#     "range_idx": None,
# }]}
# print(json.dumps(_build_tests_from_indices(_mock_tokens, _mock_plan), ensure_ascii=False, indent=2))

In [37]:
# 검증: 현재 노트북의 ocr_data를 LLM 전용(인덱스 기반) 파서에 넣어 결과 확인

print(ocr_data)

parsed = analyze_medical_ocr_data_with_llm(ocr_data)
print(json.dumps(parsed, ensure_ascii=False, indent=2))

[{'text': 'BAEK', 'confidence': 0.84}, {'text': '울', 'confidence': 0.54}, {'text': '백산동물병원', 'confidence': 0.98}, {'text': '진료기록-아전경(양희)', 'confidence': 0.92}, {'text': 'SAN', 'confidence': 0.99}, {'text': '서울특별시 강남구 역삼동725-15 Tel) 02-1644-5653', 'confidence': 0.94}, {'text': '8.64', 'confidence': 1.0}, {'text': '%', 'confidence': 1.0}, {'text': 'LYMPHO(%)', 'confidence': 0.99}, {'text': '2.496', 'confidence': 1.0}, {'text': '%', 'confidence': 0.99}, {'text': 'Mono(%)', 'confidence': 0.99}, {'text': 'EOS(%)', 'confidence': 0.96}, {'text': '0.962', 'confidence': 0.95}, {'text': '%', 'confidence': 1.0}, {'text': 'BASO (%)', 'confidence': 0.96}, {'text': '%', 'confidence': 1.0}, {'text': 'NEUT', 'confidence': 1.0}, {'text': '1.15 - 10.29', 'confidence': 0.94}, {'text': '18.065', 'confidence': 0.97}, {'text': 'KuL', 'confidence': 0.97}, {'text': 'LYMPH', 'confidence': 0.98}, {'text': '0.92-6.88', 'confidence': 0.99}, {'text': '1.78', 'confidence': 0.98}, {'text': 'KuL', 'confidence': 0.98}

In [38]:
# 비교 실행: 두 모델(gpt-4o, gpt-4.1-mini)만 단일 루프로 실행하고 비교
import time
import copy
import json

print(ocr_data)

# 입력은 동일하게 유지
_tokens = copy.deepcopy(ocr_data)

# 단일 케이스 목록 (순서 중요: 좌측이 기준처럼 표시됨)
cases = [
    ("gpt-4o", "gpt-4o"),
    ("gpt-4.1-mini", "gpt-4.1-mini"),
]

results = {}

# 두 모델을 한 번씩 실행
for label, model in cases:
    start = time.perf_counter()
    out = analyze_medical_ocr_data_with_llm(_tokens, model=model, seed=42)
    print(f"-----------{model}-----------")
    print(json.dumps(out, ensure_ascii=False, indent=2))
    dur = (time.perf_counter() - start) * 1000.0
    results[label] = {"ms": dur, "output": out}

# 간단 비교 함수들

def _normalize_test_item(t):
    return {
        "code": t.get("code"),
        "value": t.get("value"),
        "reference_min": t.get("reference_min"),
        "reference_max": t.get("reference_max"),
        "unit": t.get("unit"),
    }

norm = {k: list(map(_normalize_test_item, v["output"].get("tests", []))) for k, v in results.items()}

# 라벨 정렬을 위한 고정 폭 포맷터
label_width = max(len(lbl) for lbl, _ in cases)
fmt = lambda lbl: f"{lbl+':':<{label_width+1}}"  # 콜론 포함 정렬

# 요약은 한 번만 출력
print("=== 결과 요약 ===")
for k, _ in cases:
    v = results[k]
    print(f"{fmt(k)} {v['ms']:.1f} ms, tests={len(v['output'].get('tests', []))}")

# 단일 쌍 차이 비교 (첫 번째 vs 두 번째)
left_label, _ = cases[0]
right_label, _ = cases[1]

print(f"\n--- 차이: {fmt(left_label)} vs {fmt(right_label)} ---")
a_tests = norm.get(left_label, [])
b_tests = norm.get(right_label, [])
max_len = max(len(a_tests), len(b_tests))
diff_count = 0
for i in range(max_len):
    a_item = a_tests[i] if i < len(a_tests) else None
    b_item = b_tests[i] if i < len(b_tests) else None
    if a_item != b_item:
        diff_count += 1
        print(f"- idx {i} 다름:\n  {fmt(left_label)} {a_item}\n  {fmt(right_label)} {b_item}")
if diff_count == 0:
    print("(차이 없음)")

# 필요 시 전체 JSON 출력 (주석 해제)
print(json.dumps(results, ensure_ascii=False, indent=2))

[{'text': 'BAEK', 'confidence': 0.84}, {'text': '울', 'confidence': 0.54}, {'text': '백산동물병원', 'confidence': 0.98}, {'text': '진료기록-아전경(양희)', 'confidence': 0.92}, {'text': 'SAN', 'confidence': 0.99}, {'text': '서울특별시 강남구 역삼동725-15 Tel) 02-1644-5653', 'confidence': 0.94}, {'text': '8.64', 'confidence': 1.0}, {'text': '%', 'confidence': 1.0}, {'text': 'LYMPHO(%)', 'confidence': 0.99}, {'text': '2.496', 'confidence': 1.0}, {'text': '%', 'confidence': 0.99}, {'text': 'Mono(%)', 'confidence': 0.99}, {'text': 'EOS(%)', 'confidence': 0.96}, {'text': '0.962', 'confidence': 0.95}, {'text': '%', 'confidence': 1.0}, {'text': 'BASO (%)', 'confidence': 0.96}, {'text': '%', 'confidence': 1.0}, {'text': 'NEUT', 'confidence': 1.0}, {'text': '1.15 - 10.29', 'confidence': 0.94}, {'text': '18.065', 'confidence': 0.97}, {'text': 'KuL', 'confidence': 0.97}, {'text': 'LYMPH', 'confidence': 0.98}, {'text': '0.92-6.88', 'confidence': 0.99}, {'text': '1.78', 'confidence': 0.98}, {'text': 'KuL', 'confidence': 0.98}

todo value_confidence 도 있어야 함. confidence 낮은것은 필터링.
todo 결과값에 입출력 토큰수 나오게 해보자.